# Hunting - Automated Data Query and MDTI API and Ingestion to Custom Table

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.8<br>
__Apache Spark Version:__ 3.1<br>
__Required Packages:__ azure-monitor-query, azure-mgmt-loganalytics<br>
__Platforms Supported:__  Azure Synapse Analytics
     
__Data Source Required:__ Log Analytics custom table defined
    
### Description
This notebook provides step-by-step instructions and sample code to query various data from Azure Log Analytics and then store it back to Log Analytocs pre-defined custom table.<br>
*** Please run the cells sequentially to avoid errors.  Please do not use "run all cells". *** <br>
Need to know more about KQL? [Getting started with Kusto Query Language](https://docs.microsoft.com/azure/data-explorer/kusto/concepts/).

## Table of Contents
1. Warm-up
2. Azure Log Analytics Data Queries
3. Save result to Azure Log Analytics Custom Table

## 1. Warm-up

In [ ]:
%pip install azure.mgmt.loganalytics

In [ ]:
%pip install azure.monitor.query

In [ ]:
%pip install azure.monitor.ingestion

In [ ]:
# Load Python libraries that will be used in this notebook
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
from azure.monitor.query import LogsQueryClient, MetricsQueryClient, LogsQueryStatus
##from azure.identity.aio import DefaultAzureCredential
from azure.monitor.ingestion import LogsIngestionClient

from azure.identity import AzureCliCredential, DefaultAzureCredential, ClientSecretCredential
from azure.core.exceptions import  HttpResponseError 

from datetime import datetime, timezone, timedelta
import requests
import pandas as pd
import numpy
import json
import ipywidgets
from IPython.display import display, HTML, Markdown

In [ ]:
tenant_id = ''
subscription_id = ''
akv_name = ''
akv_link_name = ''
workspace_id = ''
client_id_name = ''
client_secret_name = ''

In [ ]:
# Parameters for provisioning resources
resource_group_name = ""
location = ""
workspace_name = ''
workspace_resource_id = '/subscriptions/{0}/resourceGroups/{1}/providers/Microsoft.OperationalInsights/workspaces/{2}'.format(subscription_id, resource_group_name, workspace_name)
dataCollection_endpoint_name = ""
dataCollection_rule_name_for_enrichment = ""
stream_name_for_enrichment = ""
immutable_rule_id_for_enrichment = ""
dce_endpoint = ''

In [ ]:
# You may need to change resource_uri for various cloud environments.
resource_uri = "https://api.loganalytics.io"
client_id = mssparkutils.credentials.getSecret(akv_name, client_id_name, akv_link_name)
client_secret = mssparkutils.credentials.getSecret(akv_name, client_secret_name, akv_link_name)

credential = ClientSecretCredential(
    tenant_id=tenant_id, 
    client_id=client_id, 
    client_secret=client_secret)
access_token = credential.get_token(resource_uri + "/.default")
token = access_token[0]

## 2. Azure Log Analytics Data Queries

In [ ]:
la_data_client = LogsQueryClient(credential=credential)

end_time =  datetime.now(timezone.utc)
start_time = end_time - timedelta(5)
query = "YOURTABLE | where TimeGenerated > ago(3d) | project TimeGenerated, Url"
query_result = la_data_client.query_workspace(
        workspace_id=workspace_id,
        query=query,
        timespan=(start_time, end_time))

if query_result.status == LogsQueryStatus.SUCCESS:
    df_la_query = pd.DataFrame(data=query_result.tables[0].rows, columns=query_result.tables[0].columns)
    print(df_la_query)

In [ ]:
# Calling Microsoft MDTI API for List, the same template can be used for calling other Azure REST APIs with different parameters.
# For different environments, such as national clouds, you may need to use different root_url, please contact with your admins.
# It can be ---.azure.us, ---.azure.microsoft.scloud, ---.azure.eaglex.ic.gov, etc.
def call_mdti_api_for_read(token, resource):
    "Calling Microsoft MDTI API"
    headers = {"Authorization": token, "content-type":"application/json" }
    root_url = "https://graph.microsoft.com"
    mdti_url_template = "{0}/beta/security/threatIntelligence/{1}"
    mdti_url = mdti_url_template.format(root_url, resource)
    print(mdti_url)
    response = requests.get(mdti_url, headers=headers, verify=True)
    return response

def get_token_for_graph():
    resource_uri = "https://graph.microsoft.com"
    client_id = mssparkutils.credentials.getSecret(akv_name, client_id_name, akv_link_name)
    client_secret = mssparkutils.credentials.getSecret(akv_name, client_secret_name, akv_link_name)

    credential = ClientSecretCredential(
        tenant_id=tenant_id, 
        client_id=client_id, 
        client_secret=client_secret)
    access_token = credential.get_token(resource_uri + "/.default")
    return access_token[0]

In [ ]:
# Calling MDTI API, hosts as example
header_token_value = "Bearer {}".format(get_token_for_graph())
response_mdti_host = call_mdti_api_for_read(header_token_value, "hosts('www.microsoft.com')")

In [ ]:
# Data process
df_host = pd.json_normalize(response_mdti_host.json())
df_merged = pd.merge(df_la_query, df_host[['id','firstSeenDateTime','registrar']], left_on='Url', right_on='id', how="outer")
df_final = df_merged.rename(columns = {'TimeGenerated': 'TimeGenerated', 'Url': 'Url', 'registrar': 'Fact'})[['TimeGenerated', 'Url', 'Fact']]

## 3. Save result to Azure Log Analytics Custom Table

In [ ]:
# function for data converting
def convert_dataframe_to_list_of_dictionaries(df, hasTimeGeneratedColumn):
    list = df.to_dict('records')

    for row in list:
        # The dataframe may have more than one datetime columns, add all datetiome columns inside this loop, to render ISO 8601
        if hasTimeGeneratedColumn and str(row['TimeGenerated']) != "NaT":
            row['TimeGenerated']= row['TimeGenerated'].strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return list

In [ ]:
# Construct data body for LA data ingestion
list_final = convert_dataframe_to_list_of_dictionaries(df_final, True)
body = list_final

In [ ]:
# Data ingestion to LA custom table
client = LogsIngestionClient(endpoint=dce_endpoint, credential=credential, logging_enable=True)

try:
    ingestion_result = client.upload(rule_id=immutable_rule_id_for_enrichment, stream_name=stream_name_for_enrichment, logs=body)
except HttpResponseError as e:
    print(f"Upload failed: {e}")

In [ ]:
ingestion_result